In [1]:
!pip install -q transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np

In [8]:
df2 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv')
#Dropping columns that do not need to be read
df2.drop(df2.columns[[4,5]], axis=1, inplace=True)
df2.head()

,Conversation ID,Source,Text,Timestamp,ID,Sentiment,new,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,user,I,2023-04-17T21:15:34.492,00036309-c615-45b0-bd95-d4669b72e66b,Neutral,00036309-c615-45b0-bd95-d4669b72e66b,Neutral
1,00036309-c615-45b0-bd95-d4669b72e66b,bot,Hmm.. sorry I couldn't find an answer for that!,2023-04-17T21:16:08.778,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
2,00036309-c615-45b0-bd95-d4669b72e66b,bot,"To have a guided conversation instead, please ...",2023-04-17T21:16:09.801,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
3,00036309-c615-45b0-bd95-d4669b72e66b,bot,Are you an existing Tower customer?,2023-04-17T21:16:10.82,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
4,00036309-c615-45b0-bd95-d4669b72e66b,user,Yes,2023-04-17T21:16:28.619,00036309-c615-45b0-bd95-d4669b72e66b,Neutral,NaN,NaN


In [9]:
# df2['Text'] = df2['Text'].fillna('')
# grouped_df = df2.groupby('ID').agg({'Text': '. '.join, 'Sentiment': 'first'}).reset_index()
# grouped_df.head()
# #len(grouped_df)

,ID,Text,Sentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,I. Hmm.. sorry I couldn't find an answer for t...,Neutral
1,000715a4-1d49-4a54-8ee8-140808202926,I have a pending claim and the company fixing ...,Neutral
2,000c670b-4513-49d4-bf3a-daa371d4ef52,Hi there. Kia Ora! 👋. I can help you navigate ...,Positive
3,000d5541-0607-463a-b05c-aeb888a69233,Hi I actually wanted to cancel my policy. I ca...,Positive
4,0016ac21-a8ae-476d-aa75-8c4b62ce175a,"Hi Charlie bot, im trying to add my airpoints ...",Positive


In [13]:
# Assuming you have imported pandas as pd and have the DataFrame df2

# Fill missing values in the 'Text' column with an empty string
df2['Text'] = df2['Text'].fillna('')

# Filter out rows where the 'Source' column is 'bot'
df2 = df2.drop(df2[df2['Source'] == 'bot'].index)

# Group by 'ID' and concatenate the 'Text' for each group separated by a period ('.')
grouped_df = df2.groupby('ID').agg({'Text': '. '.join, 'Sentiment': 'first'}).reset_index()

# Print the first few rows of the grouped DataFrame
print(grouped_df.head())

# Extract the 'Text' paragraph of the first group and store it in a variable
text_paragraph = grouped_df['Text'][0]

print(text_paragraph)


                                     ID  \
0  00036309-c615-45b0-bd95-d4669b72e66b   
1  000715a4-1d49-4a54-8ee8-140808202926   
2  000c670b-4513-49d4-bf3a-daa371d4ef52   
3  000d5541-0607-463a-b05c-aeb888a69233   
4  0016ac21-a8ae-476d-aa75-8c4b62ce175a   

                                                Text Sentiment  
0                                             I. Yes   Neutral  
1  I have a pending claim and the company fixing ...   Neutral  
2  Hi there. I hv a question do ur cover for trav...  Positive  
3           Hi I actually wanted to cancel my policy  Positive  
4  Hi Charlie bot, im trying to add my airpoints ...  Positive  
I. Yes


In [16]:
text_paragraph = grouped_df['Text'][5]
print(text_paragraph)

My insurance payment was scheduled for yesterday but it didn't come out due to the public holiday, when will this payment come out??. Would this payment usually be impacted by a public holiday? Am I still insured if it hasn't come out yet?. Terri McMeikan. 3/2/1992. terrimcmeikan92@gmail.com. Thank you. P00004356530. Ok. no. Hi Maryanne!. Thanks for coming to help. Okay cool, will it effect my cover being late?. Awesome, thank you I was quite concerned for a moment. There is enough in there for the payment to come out Tuesday
 Have a lovely afternoon


In [17]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

#ai processing the data
sentiment_pipeline(text_paragraph)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9965479969978333}]

In [4]:
#dont look at this, all the one after this is still work on progress
#anyhting after this is to be changed cuz idk wat im doing

# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

# Torch ML libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# Misc.
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Set intial variables and constants
%config InlineBackend.figure_format='retina'

# Graph Designs
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Set GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# looking at distribution
sns.countplot(df.score)
plt.xlabel(''); #use the name of the column to see distribution of sentiment data